# Install Java, Spark, and Findspark
This installs Apache Spark 2.3.2, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!pip install py4j
!pip install pyspark

     |████████████████████████████████| 204kB 43.3MB/s 
     |████████████████████████████████| 215.7MB 64kB/s 
     |████████████████████████████████| 204kB 55.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=53fa43efd68d50f78561f4969da5668de7ac38fc64910ef99f251a9199228e49
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
  Found existing installation: py4j 0.10.8.1
    Uninstalling py4j-0.10.8.1:
      Successfully uninstalled py4j-0.10.8.1


# Set Environment Variables
Set the locations where Spark and Java are installed.

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# Start a SparkSession
This will start a local Spark session.

In [0]:
import os
import time

In [0]:
# spark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction, explode, desc
from pyspark.sql.types import StringType, ArrayType
from pyspark.mllib.recommendation import ALS

In [0]:
# data science imports
import math
import numpy as np
import pandas as pd

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp /content/gdrive/My\ Drive/MscsDs/msdchallenge/kaggle_visible_evaluation_triplets.txt .

In [0]:
!cp -r /content/gdrive/My\ Drive/MscsDs/MSongsDB-master/PythonSrc .

In [0]:
# ! cd /content/gdrive/My\ Drive/MscsDs/ && tar -zxvf millionsongsubset_full.tar.gz

In [11]:
users = pd.read_table('./kaggle_visible_evaluation_triplets.txt', sep='\t', names= ['user_id','song_ids','play_count'])
users.head()

,user_id,song_ids,play_count
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1


In [17]:
nd = pd.read_csv('/content/gdrive/My Drive/MscsDs/MillionSongSubset/msd.csv')
nd=nd[['song_ids','song_titles','artist_names']]
nd.head()

,song_ids,song_titles,artist_names
0,SONHTWP12A8C142744,Vals '89,Manolis Famellos & I Podilates
1,SOPIHMB12AC468E0DD,Ghost of the Ocean (Live),Uriah Heep
2,SODRPJT12AC468DEF1,The frog song,Alain-François
3,SOLLOTO12AB01804C6,Deep Blue Sea (Daniel Rossen home recording),Grizzly Bear
4,SOCNYGS12AB01832B8,La vérité,Annie Blanchard


In [0]:
from pandas import DataFrame

temp = DataFrame(users, columns= ['user_id', 'song_ids','play_count'])
temp = pd.merge(users, nd)

In [21]:
temp

,user_id,song_ids,play_count,song_titles,artist_names
0,c34670d9c1718361feb93068a853cead3c95b76a,SOEHWGF12A6D4F8B2B,3,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean
1,c759e740af57c477fe358e62ad7b3b1f2f113a2f,SOEHWGF12A6D4F8B2B,6,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean
2,8f6e833948caf895a3281ded74ceb32783916e42,SOEHWGF12A6D4F8B2B,1,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean
3,8cd07652ccba3754502c8fa8fe809c418fdcdc8f,SOEHWGF12A6D4F8B2B,2,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean
4,563e8088cbb8affb096d2334531624a8cf7ff3d2,SOEHWGF12A6D4F8B2B,5,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean
...,...,...,...,...,...
18127,a4a85f48a7d787b6e17ed38001abcd032cb87c6a,SOKLIMD12A8C145AC3,1,Whiskey on the Fire,Aaron Watson
18128,90bac2ccc1660cf444569f3553e6b141d132b2c3,SONCEJF12AB018581C,1,Jamming,Twinkle Twinkle Little Rock Star
18129,46a363f17bd4e07d2c607779877baedafc07e7d7,SOKWNWI12A8C13383F,1,Bianca's Midnight Lullaby,Al Di Meola
18130,8ecfca10c16f046ae53634347d6b9029c40425c0,SONQKDG12A8C130D96,1,Isabel,Juana Molina


In [22]:
from sklearn.preprocessing import LabelEncoder

frame = temp
lb_make = LabelEncoder()
frame["user_id_code"] = lb_make.fit_transform(frame["user_id"])
frame[["user_id", "user_id_code"]].head(11)

lb_make2 = LabelEncoder()
frame["song_id_code"] = lb_make2.fit_transform(frame["song_ids"])
frame[["song_ids", "song_id_code"]].head(11)


,song_ids,song_id_code
0,SOEHWGF12A6D4F8B2B,253
1,SOEHWGF12A6D4F8B2B,253
2,SOEHWGF12A6D4F8B2B,253
3,SOEHWGF12A6D4F8B2B,253
4,SOEHWGF12A6D4F8B2B,253
5,SOEHWGF12A6D4F8B2B,253
6,SOEHWGF12A6D4F8B2B,253
7,SOEHWGF12A6D4F8B2B,253
8,SOEHWGF12A6D4F8B2B,253
9,SOEHWGF12A6D4F8B2B,253


In [23]:
frame

,user_id,song_ids,play_count,song_titles,artist_names,user_id_code,song_id_code
0,c34670d9c1718361feb93068a853cead3c95b76a,SOEHWGF12A6D4F8B2B,3,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean,12353,253
1,c759e740af57c477fe358e62ad7b3b1f2f113a2f,SOEHWGF12A6D4F8B2B,6,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean,12610,253
2,8f6e833948caf895a3281ded74ceb32783916e42,SOEHWGF12A6D4F8B2B,1,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean,8967,253
3,8cd07652ccba3754502c8fa8fe809c418fdcdc8f,SOEHWGF12A6D4F8B2B,2,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean,8818,253
4,563e8088cbb8affb096d2334531624a8cf7ff3d2,SOEHWGF12A6D4F8B2B,5,Hips Don't Lie (featuring Wyclef Jean),Shakira Featuring Wyclef Jean,5387,253
...,...,...,...,...,...,...,...
18127,a4a85f48a7d787b6e17ed38001abcd032cb87c6a,SOKLIMD12A8C145AC3,1,Whiskey on the Fire,Aaron Watson,10359,587
18128,90bac2ccc1660cf444569f3553e6b141d132b2c3,SONCEJF12AB018581C,1,Jamming,Twinkle Twinkle Little Rock Star,9043,703
18129,46a363f17bd4e07d2c607779877baedafc07e7d7,SOKWNWI12A8C13383F,1,Bianca's Midnight Lullaby,Al Di Meola,4427,604
18130,8ecfca10c16f046ae53634347d6b9029c40425c0,SONQKDG12A8C130D96,1,Isabel,Juana Molina,8927,725


In [0]:
export_csv = frame.to_csv (r'./triplets.csv', index = None, header=True)

In [0]:
# spark imports
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction, explode, desc
from pyspark.sql.types import StringType, ArrayType

In [0]:
spark = SparkSession \
    .builder \
    .appName("Music Recommender System") \
    .config("spark.driver.maxResultSize", "96g") \
    .config("spark.driver.memory", "96g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.master", "local[12]") \
    .getOrCreate()
# get spark context
sc = spark.sparkContext

In [27]:
df = spark.read.load('./triplets.csv', format='csv', header=True, inferSchema=True)
df.show(10, truncate=False)

+----------------------------------------+------------------+----------+--------------------------------------+-----------------------------+------------+------------+
|user_id                                 |song_ids          |play_count|song_titles                           |artist_names                 |user_id_code|song_id_code|
+----------------------------------------+------------------+----------+--------------------------------------+-----------------------------+------------+------------+
|c34670d9c1718361feb93068a853cead3c95b76a|SOEHWGF12A6D4F8B2B|3         |Hips Don't Lie (featuring Wyclef Jean)|Shakira Featuring Wyclef Jean|12353       |253         |
|c759e740af57c477fe358e62ad7b3b1f2f113a2f|SOEHWGF12A6D4F8B2B|6         |Hips Don't Lie (featuring Wyclef Jean)|Shakira Featuring Wyclef Jean|12610       |253         |
|8f6e833948caf895a3281ded74ceb32783916e42|SOEHWGF12A6D4F8B2B|1         |Hips Don't Lie (featuring Wyclef Jean)|Shakira Featuring Wyclef Jean|8967        |253   

In [0]:
# indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(df.columns)-set(['overall'])) ]
# pipeline = Pipeline(stages=indexer)
# transformed = pipeline.fit(df).transform(df)
# transformed.show(truncate=False)

In [0]:
# user = list(transformed.select('user_id_index').distinct().sort(transformed["user_id_index"])) # Get our unique customers
# song = list(transformed.select('song_ids_index').distinct().sort(transformed["song_ids_index"])) # Get our unique products that were purchased
# playCount = list(transformed.select('play_count')) # All of our purchases

# # rows = transformed.select('user_id_index').astype('category', categories = user).cat.codes 
# # # Get the associated row indices
# # cols = transformed.song_ids_index.astype('category', categories = song).cat.codes 
# # Get the associated column indices
# # playCount_sparse = sparse.csr_matrix((playCount, (user, song)), shape=(len(user), len(song)))


In [0]:
(training,test)=df.randomSplit([0.8, 0.2])

In [0]:
als=ALS(maxIter=5, regParam=0.09,rank=25,userCol="user_id_code",itemCol="song_id_code",ratingCol="play_count",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

In [30]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="play_count",predictionCol="prediction")
predictions=model.transform(test)
predictions.show(50, truncate = False)

+----------------------------------------+------------------+----------+----------------------------------------+----------------------------------------------+------------+------------+----------+
|user_id                                 |song_ids          |play_count|song_titles                             |artist_names                                  |user_id_code|song_id_code|prediction|
+----------------------------------------+------------------+----------+----------------------------------------+----------------------------------------------+------------+------------+----------+
|e9540efbbef64e43274b9243242a6b7805d9ea23|SOQHTUT12A8AE475F2|1         |Let It Go                               |Kirk Franklin with tobyMac & Sonny (of P.O.D.)|14669       |833         |0.45323807|
|50b6af13a2dab6f62d6fcb40857e6858cb1d277a|SOQHTUT12A8AE475F2|1         |Let It Go                               |Kirk Franklin with tobyMac & Sonny (of P.O.D.)|5026        |833         |0.14361864|
|6b7325774

In [0]:
rmse = evaluator.evaluate(predictions)

In [44]:
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 5.517126413712533


In [0]:
# user_recs=model.recommendForAllUsers(10).show(10, truncate = False)

+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id_index|recommendations                                                                                                                                                                                              |
+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|148          |[[132559, 128.63263], [147680, 83.976524], [66419, 82.88117], [125845, 76.72285], [82226, 76.19292], [39411, 76.14007], [162509, 75.64509], [99909, 63.91176], [98323, 63.563046], [117480, 62.751663]]      |
|463          |[[50694, 81.836334], [39411, 64.48308], [147680, 64.23534], [122151, 61.286102], [91642, 56.23076

In [0]:
recs=model.recommendForAllUsers(10).toPandas()

In [32]:
recs

,user_id_code,recommendations
0,1580,"[(226, 12.034584999084473), (990, 7.5423817634..."
1,4900,"[(226, 12.202627182006836), (339, 3.8053884506..."
2,6620,"[(226, 93.89700317382812), (643, 57.0851860046..."
3,7340,"[(704, 114.73605346679688), (226, 99.090141296..."
4,7880,"[(226, 47.703304290771484), (704, 34.140014648..."
...,...,...
13134,6438,"[(226, 40.68645477294922), (792, 13.7023382186..."
13135,15908,"[(226, 11.6093168258667), (120, 6.681258678436..."
13136,89,"[(226, 4.944281578063965), (979, 2.78582954406..."
13137,7999,"[(226, 67.30372619628906), (990, 64.1438751220..."


In [33]:
recs.shape

(13139, 2)

In [0]:
song_df = pd.read_csv('/content/gdrive/My Drive/MscsDs/MillionSongSubset/msd.csv')

In [35]:
song_df.head()

,Unnamed: 0,Unnamed: 0.1,song_ids,song_titles,song_durations,song_realease_years,artist_names,song_hotness,song_tempo,song_energies,song_danceability,song_bars,song_beats,song_time_signatures,song_tatum,song_modes,song_keys,artist_familiarty,artist_hotness
0,0,0,SONHTWP12A8C142744,Vals '89,162.58567,0,Manolis Famellos & I Podilates,NaN,94.301,0.0,0.0,[ 1.45502 3.39383 5.32826 7.25953 9.1...,[ 0.16441 0.80395 1.45502 2.09102 2.7...,3,[ 0.16441 0.48418 0.80395 1.13522 1.4...,1,0,0.228730,0.000000
1,1,1,SOPIHMB12AC468E0DD,Ghost of the Ocean (Live),214.25587,0,Uriah Heep,0.2502,149.921,0.0,0.0,[ 1.20218 2.80457 4.39266 5.98514 7.5...,[ 0.38752 0.79653 1.20218 1.60244 2.0...,4,[ 0.2523 0.38752 0.52273 ... 212.97102 2...,1,7,0.673239,0.479205
2,2,2,SODRPJT12AC468DEF1,The frog song,169.87383,0,Alain-François,NaN,136.044,0.0,0.0,[ 1.37397 4.48312 7.56804 10.65675 13.7...,[ 0.48609 0.93058 1.37397 1.81958 2.2...,7,[ 0.26328 0.48609 0.70778 0.93058 1.1...,1,2,0.380317,0.300918
3,3,3,SOLLOTO12AB01804C6,Deep Blue Sea (Daniel Rossen home recording),351.50322,2008,Grizzly Bear,NaN,85.003,0.0,0.0,[ 0.72449 3.55798 6.37712 9.20176 12.0...,[ 0.72449 1.43405 2.1445 2.85228 3.5...,4,[1.8653000e-01 3.6881000e-01 5.4665000e-01 ......,1,9,0.760636,0.547244
4,4,4,SOCNYGS12AB01832B8,La vérité,252.44689,2007,Annie Blanchard,0.0000,113.287,0.0,0.0,[2.2334000e-01 2.3324800e+00 4.4484000e+00 6.5...,[2.2334000e-01 7.5000000e-01 1.2775800e+00 1.8...,4,[2.2334000e-01 3.9890000e-01 5.7445000e-01 ......,1,5,0.479735,0.352460


In [41]:
count = 0
song_id_found = set()
for recValue in recs.values:
    for item in recValue[1]:
      row = frame.loc[frame['song_id_code'] == item.song_id_code]
      song_id = row.song_ids.values[0]
      if ( song_id not in song_id_found) and (song_df.loc[ song_df['song_ids'] == song_id].shape[0] == 1):
        rowDetails = song_df.loc[ song_df['song_ids'] == song_id]
        print(" Recommendation to user ", recValue[0], " : ")
        print("   Song: ", rowDetails.song_titles.values[0])  
        print("   Artist: ",  rowDetails.artist_names.values[0])
        song_id_found.add(song_id)
        count += 20
        
    if count > 20:
      break

 Recommendation to user  1580  : 
   Song:  Till There Was You (John Creamer & Stephane K Remix)
   Artist:  Rachael Starr
 Recommendation to user  1580  : 
   Song:  Are You There (With Another Girl)
   Artist:  Burt Bacharach
 Recommendation to user  1580  : 
   Song:  The Dynamo Of Volition [From An All Night Session]
   Artist:  Jason Mraz
 Recommendation to user  1580  : 
   Song:  La mujer que no sone (Album)
   Artist:  Ricardo Arjona
 Recommendation to user  1580  : 
   Song:  Mas P**' (America'S Mexican)
   Artist:  George Lopez
 Recommendation to user  1580  : 
   Song:  Love Ain't No Stranger (2003 Digital Remaster)
   Artist:  Whitesnake
 Recommendation to user  1580  : 
   Song:  Illuminate
   Artist:  Orbital
 Recommendation to user  1580  : 
   Song:  Song From Moulin Rouge
   Artist:  Percy Faith & His Orchestra
 Recommendation to user  1580  : 
   Song:  Camino
   Artist:  Gipsy Kings
 Recommendation to user  1580  : 
   Song:  Set The World On Fire
   Artist:  Britt N

In [42]:
print(" Recommendation to user 148 : ")
for recom in recs.query('user_id_code == 148').recommendations:
  for item in recom:
    row = frame.loc[frame['song_id_code'] == item.song_id_code]
    print("\n   Song ID: ", row.song_ids.values[0])
    song_id = row.song_ids.values[0]
    if song_df.loc[ song_df['song_ids'] == song_id].shape[0] == 1:
      rowDetails = song_df.loc[ song_df['song_ids'] == song_id]
      print("     Song: ", rowDetails.song_titles.values[0])  
      print("     Artist: ",  rowDetails.artist_names.values[0])

 Recommendation to user 148 : 

   Song ID:  SODUSRP12A6D4FA762
     Song:  Till There Was You (John Creamer & Stephane K Remix)
     Artist:  Rachael Starr

   Song ID:  SONCQTI12A8AE477B2
     Song:  Mas P**' (America'S Mexican)
     Artist:  George Lopez

   Song ID:  SOPHJIS12AC3DF5FA3
     Song:  La mujer que no sone (Album)
     Artist:  Ricardo Arjona

   Song ID:  SOCDAIR12A8C131F89
     Song:  The Dynamo Of Volition [From An All Night Session]
     Artist:  Jason Mraz

   Song ID:  SOTMGME12AB017F79B
     Song:  Are You There (With Another Girl)
     Artist:  Burt Bacharach

   Song ID:  SOLTZTZ12A58A7A84F
     Song:  Best Friends
     Artist:  Kieran

   Song ID:  SOOBUUM12A8C13696A
     Song:  Telling Me Lies
     Artist:  Dolores Keane

   Song ID:  SOGLETQ12AB0186E93
     Song:  Pose No Threat
     Artist:  Lunasicc

   Song ID:  SOGEQKA12A6D4FA27C
     Song:  Love Ain't No Stranger (2003 Digital Remaster)
     Artist:  Whitesnake

   Song ID:  SOITOFS12A81C21D00
     Song

In [45]:
print(" Recommendation to user 6438 : ")
for recom in recs.query('user_id_code == 6438').recommendations:
  for item in recom:
    row = frame.loc[frame['song_id_code'] == item.song_id_code]
    print("\n   Song ID: ", row.song_ids.values[0])
    song_id = row.song_ids.values[0]
    if song_df.loc[ song_df['song_ids'] == song_id].shape[0] == 1:
      rowDetails = song_df.loc[ song_df['song_ids'] == song_id]
      print("     Song: ", rowDetails.song_titles.values[0])  
      print("     Artist: ",  rowDetails.artist_names.values[0])

 Recommendation to user 6438 : 

   Song ID:  SODUSRP12A6D4FA762
     Song:  Till There Was You (John Creamer & Stephane K Remix)
     Artist:  Rachael Starr

   Song ID:  SOPHJIS12AC3DF5FA3
     Song:  La mujer que no sone (Album)
     Artist:  Ricardo Arjona

   Song ID:  SOGLETQ12AB0186E93
     Song:  Pose No Threat
     Artist:  Lunasicc

   Song ID:  SOVFRDB12AB017FF3A
     Song:  Got It Twisted
     Artist:  Mobb Deep

   Song ID:  SOTHZUA12A8C13268A
     Song:  All I Ask For (Album Version)
     Artist:  Ill Nino

   Song ID:  SOZAQVA12AB0183627
     Song:  Eight More Miles To Louisville
     Artist:  Jim Kweskin

   Song ID:  SOITOFS12A81C21D00
     Song:  Camino
     Artist:  Gipsy Kings

   Song ID:  SONCQTI12A8AE477B2
     Song:  Mas P**' (America'S Mexican)
     Artist:  George Lopez

   Song ID:  SOVMKKT12A58A7F0CC
     Song:  Love Unlimited (Explicit)
     Artist:  Fun Lovin' Criminals

   Song ID:  SOCDAIR12A8C131F89
     Song:  The Dynamo Of Volition [From An All Night 

In [0]:
rec_item = model.recommendForAllItems(10).toPandas()

In [46]:
rec_item

,song_id_code,recommendations
0,471,"[(6241, 18.191701889038086), (424, 14.06768226..."
1,463,"[(424, 17.405658721923828), (1907, 15.71860694..."
2,833,"[(12902, 39.05113220214844), (117, 29.65908813..."
3,496,"[(6241, 13.5421142578125), (6223, 12.761930465..."
4,148,"[(6241, 227.50213623046875), (13241, 131.88528..."
...,...,...
1204,401,"[(6241, 36.72122573852539), (424, 23.037067413..."
1205,422,"[(6241, 21.395238876342773), (424, 19.07993888..."
1206,517,"[(12902, 30.867374420166016), (117, 23.4435768..."
1207,1138,"[(12902, 35.97333908081055), (424, 33.69433212..."


In [0]:
# print(" Recommendation for 1138 : ")
# for recom in rec_item.query('song_id_code == 1138').recommendations:
#   for item in recom:
#     row = frame.loc[frame['user_id_code'] == item.user_id_code]
#     print("\n   user_id ID: ", row.user_id.values[0])
#     user_id = row.user_id.values[0]
#     if frame.loc[ frame['user_id'] == user_id].shape[0] > 0:
#       rowDetails = frame.loc[ frame['user_id'] == song_id]
#       print(rowDetails.values)
#       print("     Song: ", rowDetails.song_titles.values[0])  
#       print("     Artist: ",  rowDetails.artist_names.values[0])